## RF

----

### 1. Import Modules

In [2]:
import numpy as np
import pandas as pd
import cimcb as cb
from sklearn.model_selection import train_test_split

print('All packages successfully loaded')

Using TensorFlow backend.


All packages successfully loaded


### 2. Load data and peak sheet

In [3]:
home = 'data/' 
file = 'GastricCancer_NMR.xlsx' 

DataTable,PeakTable = cb.utils.load_dataXL(home + file, DataSheet='Data', PeakSheet='Peak') 

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 140 TOTAL PEAKS: 149
Done!


### 3. Extract X & Y

In [4]:
# Clean PeakTable
RSD = PeakTable['QC_RSD']   
PercMiss = PeakTable['Perc_missing']  
PeakTableClean = PeakTable[(RSD < 20) & (PercMiss < 10)]   

# Select subset of Data for the PLS-DA model
DataTable2 = DataTable[(DataTable.Class == "GC") | (DataTable.Class == "HE")]

# Create a Binary Y vector for stratifiying the samples
Outcomes = DataTable2['Class']                                  
Y = [1 if outcome == 'GC' else 0 for outcome in Outcomes]         
Y = np.array(Y)   

# Extract and scale the metabolite data from the DataTable
peaklist = PeakTable['Name']                           
XT = DataTable2[peaklist]                                    
XTlog = np.log(XT)                                          
XTscale = cb.utils.scale(XTlog, method='auto')              
XTknn = cb.utils.knnimpute(XTscale, k=3)     

### 4. Hyperparameters optimisation

In [12]:
# param_dict
depth = list(range(1,11))
feat = list(range(1,11))
param_dict = dict(max_depth=depth, max_features=feat, criterion='gini', min_samples_split=2, min_samples_leaf=1, max_leaf_nodes=None, n_estimators=10)


# Initalise
cv = cb.cross_val.kfold(model=cb.model.RF,                      
                                X=XTknn,                                 
                                Y=Y,                               
                                param_dict=param_dict,                   
                                folds=5,
                                n_mc=1)                                

# Run and plot
cv.run()  
cv.plot(metric='auc', ci=95, scale=1.3, color_beta=5)

Number of cores set to: 8
Running ...


100%|██████████| 100/100 [00:01<00:00, 71.66it/s]


Time taken: 0.03 minutes with 8 cores
Done!


Loading BokehJS ...

### 6. Get Model and Evaluate

In [13]:
# 6. Train, test and evaluate model

# Train model
model = cb.model.RF(max_depth=2, max_features=9, criterion='gini', min_samples_split=2, min_samples_leaf=1, max_leaf_nodes=None, n_estimators=10)
model.train(XTknn, Y)
model.test(XTknn)

# Evaluate Ypred against Ytest
model.evaluate(cutoffscore=0.5, plot_median=False) 

Loading BokehJS ...

In [14]:
# Bootstrap evaluate

model.booteval(XTknn, Y, cutoffscore=0.5, bootnum=100) 

100%|██████████| 100/100 [00:01<00:00, 62.94it/s]


Loading BokehJS ...

### 7. Save table

In [15]:
home = 'tables/'
file = 'RF_GastricCancer_NH.xlsx'

model.save_table(home + file)

Done! Saved table as tables/RF_GastricCancer_NH.xlsx
